In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from random import random
import time
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
browser = webdriver.Safari()
base_url = 'https://www.glassdoor.com/blog/tag/job-search/'
browser.get(base_url)

In [6]:
keyword = 'product manager'

query = browser.find_element_by_id("sc.keyword")
query.click()
query.clear()
time.sleep(random()*1)
query.send_keys(keyword)
time.sleep(random()*0.1)
return_button = browser.find_element_by_xpath('//*[@id="HeroSearchButton"]')
return_button.click()
time.sleep(random() * 0.1)

links = set()

In [7]:
pageToGo = 30

for i in range(pageToGo):
    
    #try and catch clauses
    try:
        close = browser.find_element_by_class_name('modal_closeIcon')
        close.click()
        close.click()
    except:
        None
    try:
        close = browser.find_element_by_class('qual_x_close')
        close.click()
        close.click()
    except:
        None
    
    #get job_link
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    grids = soup.find('ul', class_='jlGrid hover').find_all('a', href=True)
    for g in grids:
        links.add(g['href'])
    
    #go to next page
    nextButton = browser.find_element_by_class_name('next')   
    time.sleep(2)
    nextButton.click()
    nextButton.click()
    time.sleep(2)

browser.close()

In [8]:
f = open("product_manager_links.txt", "w")
for link in links:
    f.write(link)
    f.write('\n')
f.close()

In [9]:
links = list(links)
web_pages = []
count=0

In [10]:
len(links)

958

In [11]:
import requests

#for l in links[474:]:
for l in links:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    contents = requests.get(l, headers = headers)
    time.sleep(5+random()*3)
    if  contents.status_code != requests.codes.ok:
        print('This is not OK. 403')
        print('Count: ', count)
        print('Will Try again')
        time.sleep(20)
        contents = requests.get(l, headers = headers)
        if  contents.status_code != requests.codes.ok:
            print('failed')
            break
    if count % 10 == 0:
        print('currently on #', count, 'status: ', contents)
        if count % 100 == 0:
            temp = BeautifulSoup(contents.text, 'html.parser')
            jobDescription = soup.find_all('div', id='JobDescriptionContainer')
            if len(jobDescription) < 1:
                print('No job description...')
                print('Check carefully')
    web_pages.append(contents.text)
    count += 1

currently on # 0 status:  <Response [200]>
currently on # 10 status:  <Response [200]>
currently on # 20 status:  <Response [200]>
currently on # 30 status:  <Response [200]>
currently on # 40 status:  <Response [200]>
currently on # 50 status:  <Response [200]>
currently on # 60 status:  <Response [200]>
currently on # 70 status:  <Response [200]>
currently on # 80 status:  <Response [200]>
currently on # 90 status:  <Response [200]>
currently on # 100 status:  <Response [200]>
currently on # 110 status:  <Response [200]>
currently on # 120 status:  <Response [200]>
currently on # 130 status:  <Response [200]>
currently on # 140 status:  <Response [200]>
currently on # 150 status:  <Response [200]>
currently on # 160 status:  <Response [200]>
currently on # 170 status:  <Response [200]>
currently on # 180 status:  <Response [200]>
currently on # 190 status:  <Response [200]>
currently on # 200 status:  <Response [200]>
currently on # 210 status:  <Response [200]>
currently on # 220 st

In [12]:
#Save original html file to archive. #too big, do not include it in github. Share using drive.
import os
os.makedirs(os.path.dirname('./archive_web_page/' + keyword.replace(' ', '_') + '/'), exist_ok=True)

for i, web_page in enumerate(web_pages):
    soup = BeautifulSoup(web_page, 'html.parser')
    filename = './archive_web_page/' + keyword.replace(' ', '_') + '/' + str(i) + '_' + str(soup.title.text).replace(" ", "_").replace('/', '_') + '.html'
    file = open(filename, "w")
    file.write(web_page)
    file.close()
print('Write Complete.')

Write Complete.


In [13]:
data = []

In [14]:
for i, web_page in enumerate(web_pages):
    print('---------Currently converting web page # ', i)
    soup = BeautifulSoup(web_page, 'html.parser')
    try:
        company_name = soup.find('div', class_='css-16nw49e e11nt52q1').find(text=True, recursive=False)
    except:
        print('No company name')
        company_name = ''
    try:
        rating = soup.find('span', class_='css-1pmc6te e11nt52q3').find(text=True, recursive=False)
    except:
        print('No rating')
        rating = ''
    try:
        title = soup.find('div', class_='css-17x2pwl e11nt52q5').find(text=True, recursive=False)
    except:
        print('No title')
        title = ''
    try:
        location = soup.find('div', class_='css-13et3b1 e11nt52q2').find(text=True, recursive=False)
    except:
        print('No location')
        location = ''
        
    try:
        estimate_salary_range = soup.find('span', class_='small css-10zcshf e1v3ed7e1').find(text=True, recursive=False)
    except:
        print('no estimate_salary_range')
        estimate_salary_range = '-1'
    try:
        estimate_salary_point = soup.find('h2', class_='css-mena9f e10nsqxw5').find(text=True, recursive=False)
    except:
        print('no estimate_salary_point')
        estimate_salary_point = '-1'
    try:
        job_description = soup.find('div', class_='desc css-58vpdc ecgq1xb3').text
    except:
        print('no job_description')
        job_description = ''
    
    try:
        company_link = soup.find('a', class_='mt-0 css-1sltc87 epu0oo21')['href']
    except:
        print('no company link')
        company_link = ''
        
    entry = {
        'title': str(soup.title.text).replace(" ", "_"),
        'company_name': company_name,
        'rating': rating,
        'title': title,
        'location': location,
        'estimate_salary_range': estimate_salary_range,
        'estimate_salary_point': estimate_salary_point,
        'job_description': job_description,
        'company_link': company_link
        }
    
    data.append(entry)

---------Currently converting web page #  0
---------Currently converting web page #  1
no estimate_salary_point
---------Currently converting web page #  2
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  3
---------Currently converting web page #  4
---------Currently converting web page #  5
---------Currently converting web page #  6
---------Currently converting web page #  7
---------Currently converting web page #  8
---------Currently converting web page #  9
---------Currently converting web page #  10
---------Currently converting web page #  11
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  12
---------Currently converting web page #  13
---------Currently converting web page #  14
---------Currently converting web page #  15
---------Currently converting web page #  16
---------Currently converting web page #  17
---------Currently converting web page #  18
---------Currently converting 

---------Currently converting web page #  151
---------Currently converting web page #  152
---------Currently converting web page #  153
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  154
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  155
---------Currently converting web page #  156
---------Currently converting web page #  157
no company link
---------Currently converting web page #  158
No rating
no estimate_salary_range
no estimate_salary_point
no company link
---------Currently converting web page #  159
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  160
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  161
---------Currently converting web page #  162
---------Currently converting web page #  163
---------Currently converting web page #  164
no estimate_salary_range
no estimate_salary_point
no company lin

---------Currently converting web page #  288
---------Currently converting web page #  289
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  290
---------Currently converting web page #  291
---------Currently converting web page #  292
---------Currently converting web page #  293
---------Currently converting web page #  294
No rating
no estimate_salary_range
no estimate_salary_point
no company link
---------Currently converting web page #  295
---------Currently converting web page #  296
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  297
---------Currently converting web page #  298
---------Currently converting web page #  299
---------Currently converting web page #  300
---------Currently converting web page #  301
---------Currently converting web page #  302
No rating
no estimate_salary_range
no estimate_salary_point
no company link
---------Currently converting web page #  303
---------Cur

---------Currently converting web page #  427
---------Currently converting web page #  428
---------Currently converting web page #  429
---------Currently converting web page #  430
---------Currently converting web page #  431
---------Currently converting web page #  432
---------Currently converting web page #  433
---------Currently converting web page #  434
no company link
---------Currently converting web page #  435
No rating
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  436
---------Currently converting web page #  437
---------Currently converting web page #  438
---------Currently converting web page #  439
---------Currently converting web page #  440
---------Currently converting web page #  441
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  442
---------Currently converting web page #  443
---------Currently converting web page #  444
---------Currently converting web page #  445


---------Currently converting web page #  566
---------Currently converting web page #  567
---------Currently converting web page #  568
---------Currently converting web page #  569
---------Currently converting web page #  570
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  571
No rating
no estimate_salary_point
no company link
---------Currently converting web page #  572
---------Currently converting web page #  573
---------Currently converting web page #  574
no estimate_salary_point
---------Currently converting web page #  575
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  576
---------Currently converting web page #  577
---------Currently converting web page #  578
---------Currently converting web page #  579
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  580
---------Currently converting web page #  581
no estimate_salary_range
no estimate_s

---------Currently converting web page #  707
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  708
---------Currently converting web page #  709
---------Currently converting web page #  710
---------Currently converting web page #  711
---------Currently converting web page #  712
---------Currently converting web page #  713
---------Currently converting web page #  714
No rating
no estimate_salary_range
no estimate_salary_point
no company link
---------Currently converting web page #  715
---------Currently converting web page #  716
No rating
no estimate_salary_range
no estimate_salary_point
no company link
---------Currently converting web page #  717
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  718
---------Currently converting web page #  719
---------Currently converting web page #  720
---------Currently converting web page #  721
---------Currently converting web page #  722
no estimate_

---------Currently converting web page #  844
---------Currently converting web page #  845
---------Currently converting web page #  846
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  847
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  848
---------Currently converting web page #  849
---------Currently converting web page #  850
---------Currently converting web page #  851
---------Currently converting web page #  852
No rating
no estimate_salary_range
no estimate_salary_point
no company link
---------Currently converting web page #  853
---------Currently converting web page #  854
---------Currently converting web page #  855
---------Currently converting web page #  856
---------Currently converting web page #  857
no estimate_salary_range
no estimate_salary_point
---------Currently converting web page #  858
---------Currently converting web page #  859
no estimate_salary_range
no estimate_s

In [15]:
print(data[0].keys())

dict_keys(['title', 'company_name', 'rating', 'location', 'estimate_salary_range', 'estimate_salary_point', 'job_description', 'company_link'])


In [16]:
df = pd.DataFrame(data, columns = data[0].keys())
print(df.head())

                                   title             company_name rating  \
0              Technical Project Manager                   Qualys    3.0   
1           Assistant Grocery Manager FT              Lowes Foods    3.4   
2             Manager, Pharmacy Programs  Christian Care Ministry    3.2   
3            Engineering Project Manager       Pentastar Aviation    3.4   
4  Program Manager, Brain Injury Program                 Bancroft    3.6   

          location estimate_salary_range estimate_salary_point  \
0  Foster City, CA           $67K-$118K                $90,215   
1    Southport, NC               $7-$11                     -1   
2    Melbourne, FL                    -1                    -1   
3    Waterford, MI            $53K-$96K                $73,672   
4   Toms River, NJ            $30K-$39K                $33,369   

                                     job_description  \
0  Description\n\nThe Technical Project Manager w...   
1  To gain the skills, knowledge, 

In [17]:
df_cleaned = df.drop_duplicates()
print("length of original dataset: ",len(df))
print("length of cleaned dataset: ",len(df_cleaned))

length of original dataset:  958
length of cleaned dataset:  592


In [18]:
df_cleaned.to_csv('glassdoor_data_pm.csv', index=False)